In [1]:
import sys

import os
import rasterio as rio
import msoffcrypto
import numpy as np
import json
import webbrowser
import requests
import pandas as pd
import time
from tqdm import tqdm

In [4]:
import msoffcrypto
import pandas as pd
from io import BytesIO


file_name = "/home/martin/zonasVerdes/input/direcciones_y_coordenadas_protegido.xlsx"

password = "CLM_DES"

with open(file_name, "rb") as f:
    encrypted = msoffcrypto.OfficeFile(f)
    encrypted.load_key(password=password)
    decrypted = BytesIO()
    encrypted.decrypt(decrypted)

# Carga el Excel desencriptado
excel = pd.ExcelFile(decrypted)
sheet_names = excel.sheet_names
print(f"Hojas disponibles: {sheet_names}")

Hojas disponibles: ['ORENSE', 'PONTEVEDRA', 'BARCELONA', 'ZARAGOZA', 'TERUEL', 'VIZCAYA', 'SEVILLA', 'LEON', 'SEGOVIA', 'TUDELA', 'ZAMORA', 'VALLADOLID', 'SALAMANCA', 'GERONA', 'MALLORCA', 'MENORCA']


In [ ]:
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
import textwrap

def get_buffered_bounds(df, lon_col='long', lat_col='lat', buffer_m=500):
    df = df.copy()

    # Reemplaza espacios y strings vacíos por NaN
    df[lon_col] = df[lon_col].replace(r'^\s*$', np.nan, regex=True)
    df[lat_col] = df[lat_col].replace(r'^\s*$', np.nan, regex=True)

    # Convierte a numérico, convirtiendo valores no numéricos a NaN
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')

    # Elimina filas sin coordenadas válidas
    df = df.dropna(subset=[lon_col, lat_col])

    if df.empty:
        raise ValueError("No hay coordenadas válidas en esta hoja.")

    # Resto igual
    points = [Point(lon, lat) for lon, lat in zip(df[lon_col], df[lat_col])]

    proj_wgs84 = pyproj.CRS('EPSG:4326')
    proj_utm = pyproj.CRS.from_user_input("EPSG:25830")
    project = pyproj.Transformer.from_crs(proj_wgs84, proj_utm, always_xy=True).transform
    project_back = pyproj.Transformer.from_crs(proj_utm, proj_wgs84, always_xy=True).transform

    multipoint = None
    for p in points:
        projected = transform(project, p)
        buffered = projected.buffer(buffer_m)
        multipoint = buffered if multipoint is None else multipoint.union(buffered)

    bounds = transform(project_back, multipoint).bounds
    return bounds

def generar_codigo_gee(nombre_hoja, bounds, fecha_inicio, fecha_fin):
    l, b, r, t = bounds

    # Si las coordenadas parecen estar en formato [lat, lon], se corrigen a [lon, lat]
    if all(-90 <= x <= 90 for x in (l, r, b, t)):
        print(f"[AVISO] Invirtiendo coordenadas para {nombre_hoja}: asumido orden [lat, lon]")
        l, b, r, t = b, l, t, r

    codigo = f"""
    var {nombre_hoja.lower()} = ee.Geometry.Polygon(
        [[[{l:.4f}, {b:.4f}],
          [{l:.4f}, {t:.4f}],
          [{r:.4f}, {t:.4f}],
          [{r:.4f}, {b:.4f}],
          [{l:.4f}, {b:.4f}]]]
    );
    var fechaInicio = '{fecha_inicio}';
    var fechaFin = '{fecha_fin}';

    var coleccion = ee.ImageCollection("COPERNICUS/S2")
      .filterBounds({nombre_hoja.lower()})
      .filterDate(fechaInicio, fechaFin)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10));

    var coleccionNDVI = coleccion.map(function(imagen) {{
      var ndvi = imagen.normalizedDifference(['B8', 'B4']).rename('NDVI');
      return ndvi.copyProperties(imagen, imagen.propertyNames());
    }});

    var ndviPromedio = coleccionNDVI.mean();

    var parametrosVisualizacion = {{
      min: 0,
      max: 1,
      palette: ['white', 'green']
    }};
    Map.centerObject({nombre_hoja.lower()}, 10);
    Map.addLayer(ndviPromedio.clip({nombre_hoja.lower()}), parametrosVisualizacion, 'NDVI Promedio');

    Export.image.toDrive({{
      image: ndviPromedio.clip({nombre_hoja.lower()}),
      description: 'NDVI_{fecha_inicio}_{nombre_hoja}',
      folder: 'EarthEngine',
      fileNamePrefix: 'NDVI_{fecha_inicio}_{nombre_hoja}',
      scale: 10,
      region: {nombre_hoja.lower()},
      maxPixels: 1e13
    }});
    """
    return textwrap.dedent(codigo)


# Ejemplo: para todas las hojas
fecha_inicio = '2023-05-01'
fecha_fin = '2023-07-31'
codigos = {}

for hoja in sheet_names:
    print(hoja)
    df = excel.parse(hoja)
    bounds = get_buffered_bounds(df)
    codigos[hoja] = generar_codigo_gee(hoja, bounds, fecha_inicio, fecha_fin)

# Mostrar un ejemplo
print(codigos[sheet_names[0]])

In [ ]:
#7,8,9,10,11,12,14,15
print(codigos[sheet_names])

In [2]:
import os
import io
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point, mapping
import msoffcrypto
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import tempfile
import pyproj
from shapely.ops import transform

def cargar_excel_protegido(ruta_excel, password):
    with open(ruta_excel, 'rb') as f:
        archivo = msoffcrypto.OfficeFile(f)
        archivo.load_key(password=password)
        excel_desprotegido = io.BytesIO()
        archivo.decrypt(excel_desprotegido)
        excel_desprotegido.seek(0)
    return pd.ExcelFile(excel_desprotegido)

def buffer_en_metros(punto, metros, crs_actual):
    # CRS métrico: pseudo‐Mercator
    crs_metrico = pyproj.CRS("EPSG:3857")
    # transformador → métrico
    to_m = pyproj.Transformer.from_crs(crs_actual, crs_metrico, always_xy=True).transform
    # transformador ← de métrico de nuevo a ráster
    to_raster = pyproj.Transformer.from_crs(crs_metrico, crs_actual, always_xy=True).transform

    punto_m = transform(to_m, punto)                  # punto en metros
    buffer_m = punto_m.buffer(metros)                 # círculo de metros
    buffer_geo = transform(to_raster, buffer_m)       # polígono de vuelta a grados
    return buffer_geo

def extraer_estadisticas_raster(ruta_raster, punto, radios=[100, 250, 500]):
    resultados = {}

    with rasterio.open(ruta_raster) as src:
        crs_raster = src.crs

        for radio in radios:
            try:
                # Si el CRS del ráster está en grados (EPSG:4326), se reproyecta
                if crs_raster.to_epsg() == 4326:
                    # Reproyectamos el punto a UTM zona 29N (válido para Galicia)
                    punto_metrico = gpd.GeoSeries([punto], crs="EPSG:4326").to_crs("EPSG:25829").iloc[0]
                    buffer_metrico = punto_metrico.buffer(radio)

                    # Reproyectamos el buffer de vuelta a EPSG:4326
                    buffer = gpd.GeoSeries([buffer_metrico], crs="EPSG:25829").to_crs("EPSG:4326").iloc[0]
                else:
                    # El CRS ya está en metros, buffer directo
                    buffer = punto.buffer(radio)

                # Extraer datos del ráster dentro del buffer
                out_image, _ = mask(src, [mapping(buffer)], crop=True)
                datos = out_image[0].astype('float')
                datos[datos == src.nodata] = np.nan
                datos = datos[~np.isnan(datos)]

                if len(datos) == 0:
                    stats = {f'{stat}_{radio}m': None for stat in ['mean','max','min', 'median', 'q25', 'q75', 'std', 'iqr']}
                else:
                    q25 = np.percentile(datos, 25)
                    q75 = np.percentile(datos, 75)
                    stats = {
                        f'mean_{radio}m': np.mean(datos),
                        f'max_{radio}m': max(datos),
                        f'min_{radio}m': min(datos),
                        f'median_{radio}m': np.median(datos),
                        f'q25_{radio}m': q25,
                        f'q75_{radio}m': q75,
                        f'std_{radio}m': np.std(datos),
                        f'iqr_{radio}m': q75 - q25,
                    }

                resultados.update(stats)

            except Exception as e:
                print(f"Error procesando radio {radio} m para punto {punto}: {e}")
                resultados.update({f'{stat}_{radio}m': None for stat in ['mean','max', 'min', 'median', 'q25', 'q75', 'std', 'iqr']})

    return resultados

def procesar_hoja(args):
    hoja, df, raster_path, output_path, buffers = args
    print(f"[+] Procesando hoja: {hoja}")

    if not os.path.exists(raster_path):
        print(f"[Aviso] Raster no encontrado para hoja '{hoja}', se omite.")
        return None

    if 'lat' not in df.columns or 'long' not in df.columns:
        print(f"[Aviso] Hoja '{hoja}' sin columnas lat/long válidas.")
        return None

    try:
        with rasterio.open(raster_path) as src:
            crs_raster = src.crs
    except Exception as e:
        print(f"[Error] No se pudo abrir el raster '{raster_path}': {e}")
        return None

    resultados = []
    for _, fila in df.iterrows():
        lat = fila.get('lat')
        lon = fila.get('long')

        if pd.isnull(lat) or pd.isnull(lon):
            resultados.append({f'{stat}_{r}m': None for r in buffers for stat in ['mean','max', 'min', 'median', 'q25', 'q75', 'std', 'iqr']})
            continue

        try:
            lat = float(lat)
            lon = float(lon)
        except ValueError:
            resultados.append({f'{stat}_{r}m': None for r in buffers for stat in ['mean', 'max', 'min', 'median', 'q25', 'q75', 'std', 'iqr']})
            continue

        try:
            punto = Point(lat,lon)
            stats = extraer_estadisticas_raster(raster_path,punto, buffers)
            resultados.append(stats)
        except Exception as e:
            print(f"[Error] Al procesar punto ({lat}, {lon}) en hoja '{hoja}': {e}")
            resultados.append({f'{stat}_{r}m': None for r in buffers for stat in ['mean', 'max', 'min', 'median', 'q25', 'q75', 'std', 'iqr']})

    df_resultado = pd.concat([df, pd.DataFrame(resultados)], axis=1)
    hoja_path = os.path.join(output_path, f"{hoja}.xlsx")
    df_resultado.to_excel(hoja_path, index=False)
    print(f"Hoja '{hoja}' procesada.")
    return hoja_path, hoja

def procesar_excel_paralelo(ruta_excel, password, carpeta_rasters, salida_excel, buffers,max_workers=4):
    xls = cargar_excel_protegido(ruta_excel, password)
    hojas_y_dfs = [(hoja, xls.parse(hoja)) for hoja in xls.sheet_names]

    with tempfile.TemporaryDirectory() as temp_dir:
        args = [
            (hoja, df, os.path.join(carpeta_rasters, f"{hoja}.tif"), temp_dir, buffers)
            for hoja, df in hojas_y_dfs
        ]

        print(f"Procesando {len(args)} hojas en paralelo...")
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            resultados = list(tqdm(executor.map(procesar_hoja, args), total=len(args)))

        writer = pd.ExcelWriter(salida_excel, engine='openpyxl')
        for res in resultados:
            if res:
                hoja_path, hoja = res
                df_temp = pd.read_excel(hoja_path)
                df_temp.to_excel(writer, sheet_name=hoja, index=False)
        writer.close()
        print(f"Todas las hojas combinadas en: {salida_excel}")


In [4]:
procesar_excel_paralelo(
    ruta_excel='/home/martin/zonasVerdes/input/direcciones_y_coordenadas_protegido.xlsx',
    password='CLM_DES',
    carpeta_rasters='/home/martin/zonasVerdes/input/EarthEngine/EarthEngine',
    salida_excel='/home/martin/zonasVerdes/input/resultados_ndvi.xlsx',
    buffers = [100,250,500,1000],
    max_workers=6  # Ajusta según tu CPU
)


Procesando 16 hojas en paralelo...


  0%|                                                    | 0/16 [00:00<?, ?it/s]

[+] Procesando hoja: ORENSE
[+] Procesando hoja: PONTEVEDRA[+] Procesando hoja: BARCELONA
[+] Procesando hoja: TERUEL[+] Procesando hoja: ZARAGOZA[+] Procesando hoja: VIZCAYA



Error procesando radio 100 m para punto POINT (0 0): Input shapes do not overlap raster.
Error procesando radio 250 m para punto POINT (0 0): Input shapes do not overlap raster.
Error procesando radio 500 m para punto POINT (0 0): Input shapes do not overlap raster.
Error procesando radio 1000 m para punto POINT (0 0): Input shapes do not overlap raster.
Hoja 'TERUEL' procesada.
[+] Procesando hoja: SEVILLA
Hoja 'ORENSE' procesada.


  6%|██▊                                         | 1/16 [00:04<01:00,  4.05s/it]

[+] Procesando hoja: LEON
Hoja 'LEON' procesada.
[+] Procesando hoja: SEGOVIA
Hoja 'PONTEVEDRA' procesada.


...

Hoja 'SALAMANCA' procesada.


 81%|██████████████████████████████████▉        | 13/16 [00:22<00:04,  1.49s/it]

Hoja 'GERONA' procesada.


100%|███████████████████████████████████████████| 16/16 [00:23<00:00,  1.44s/it]


Todas las hojas combinadas en: /home/martin/zonasVerdes/input/resultados_ndvi.xlsx


In [ ]:
import rasterio
with rasterio.open('/home/martin/zonasVerdes/input/EarthEngine/EarthEngine/ORENSE.tif') as src:
    print(src.crs)
    print(src.bounds)